<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">

# Evaluación de métodos de clasificación para la base de datos *Fetal Health*

Este *Notebook* contiene el trabajo necesario para cargar el fichero *.csv* de la base de datos de **Kaggle** sobre *salud fetal*, así como para aplicar diferentes tipos de métodos de clasificación sobre dicha base de datos.

## Autor
- Pedro Latorre Carmona

### Curso
- 2022-2023

**Kaggle** es, digamos, un repositorio, donde podemos encontrar bases de datos, así como diferentes tipos de métodos (código), para tareas que pueden ir desde la clasificación, regresión, por citar sólo dos ejemplos:

https://www.kaggle.com/

Dentro de **Kaggle**, vamos a trabajar con la base de datos de **Fetal Health**, la cual puede encontrarse en:

https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification

---
Tal y como se establece en esta página:

"La reducción de la mortalidad infantil se refleja en varios de los Objetivos de Desarrollo Sostenible de las Naciones Unidas y es un indicador clave del progreso humano. La ONU espera que para 2030, los países pongan fin a las muertes prevenibles de recién nacidos y niños menores de 5 años, con el objetivo de reducir la mortalidad de menores de 5 años al menos a 25 por cada 1000 nacidos vivos.

Paralelamente a la noción de mortalidad infantil está, por supuesto, la mortalidad materna, que representa 295000 muertes durante y después del embarazo y el parto (a fecha de $2017$). La gran mayoría de estas muertes ($94\%$) ocurrieron en entornos de bajos recursos y la mayoría podría haberse evitado.

A la luz de lo mencionado anteriormente, los cardiotocogramas (CTG) son una opción simple y económicamente accesible para evaluar la salud fetal, lo que permite a los profesionales de la salud tomar medidas para prevenir la mortalidad infantil y materna. El equipo en sí funciona enviando pulsos de ultrasonido y leyendo su respuesta, establciendo la frecuencia cardíaca fetal (FCF), los movimientos fetales, las contracciones uterinas y más."

---
### Datos

Este conjunto de datos contiene $2126$ registros de características extraídas de exámenes de cardiotocograma, que luego fueron clasificados por tres obstetras expertos, en $3$ clases:

1. Normal
2. Sospechosa
3. Patológica

Esta base de datos es en realidad un fichero **.csv** que tiene una tabla en la que cada fila es un **dato** asociado a un **ejemplo**. Dentro de cada fila (data), tenemos un conjunto de **atributos** o características, que conforman el vector con el que describimos dicho dato (denominado **vector de características**).

---
### Objetivo

El objetivo del trabajo a continuación es crear el conjunto de datos $(\mathbf{X,Y})$, generar los conjuntos de **entrenamiento** y **test** y aplicar dos métodos de clasificación:

1. Support Vector Machines (SVM)
2. Multilayer perceptron

Se tendrán que mostrar los resultados de clasificación de diferentes formas, y analizarlos.

---
Para el método de clasificación de **Support Vector Machines** (SVMs), se dará en clase una pequeña introducción, aunque se puede encontrar información muy fácilmente, ya que es un método muy usado:

https://en.wikipedia.org/wiki/Support_vector_machine

El clasificador denominado **Perceptrón multicapa** (**Multilayer perceptron**), no lo veremos, simplemente lo usaremos, aunque se puede encontrar información, por ejemplo, en (por ejemplo):

https://es.wikipedia.org/wiki/Perceptr%C3%B3n_multicapa

## Carga de la base de datos y aplicación de los métodos de clasificación


Para gestionar los datos en el fichero **.csv**, vamos a utilizar la estructura de datos de **Pandas dataframe**, cuyos detalles pueden encontrarse en:

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

En concreto, vamos a utilizar la opción que nos permite leer ficheros **.csv**, y que se encuentra en:

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [12]:
'''
Importación de librerías
'''
import pandas as pd
import numpy as np
import pickle
import os

from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline

In [13]:
'''
Datos
'''

path = "./FetalData"
filename = path + os.sep + "fetal_health.csv"

### Carga del fichero csv como un *data frame*

In [14]:
# Utilizad la opción "pd.read_csv"
df = pd.read_csv(filename)

In [15]:
# Visualizad el "data frame" usando la opción "display"
display(df)

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,2.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,2.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,2.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,2.0


## Creación de los conjuntos **X** e **y**

Una vez creado el data frame, tenéis que crear los conjuntos $\mathbf{X}$ e $\mathbf{Y}$, del que luego se crea sus correspondientes conjuntos de **entrenamiento** y **test**. 

In [16]:
y = df.fetal_health.values.astype(int)


caract_cols = ["baseline value", "accelerations", "fetal_movement", 
               "uterine_contractions", "light_decelerations", 
               "severe_decelerations", "prolongued_decelerations",
               "abnormal_short_term_variability", "mean_value_of_short_term_variability",
               "percentage_of_time_with_abnormal_long_term_variability", "mean_value_of_long_term_variability", 
               "histogram_width", "histogram_min", "histogram_max", "histogram_number_of_peaks", 
               "histogram_number_of_zeroes", "histogram_mode", "histogram_mean", 
               "histogram_median", "histogram_variance", "histogram_tendency", "fetal_health"]

X_all = df[caract_cols].values

In [17]:
print(X_all.shape)

(2126, 22)


In [19]:
'''
Listado de datos y nombres
'''

datasets = [(X_all,y)]
dataset_names = ["Data All"]

In [25]:
'''
Definición del espacio de búsqueda para la optimización de los parámetros de SVM
'''

# Para definir el rango de "C" y de "gamma", tenéis que usar la opción "np.logspace", cubriendo, para "C", 
# desde 1.0e-2 hasta 1.0e+10, y para "gamma", desde 1.0e-9, hasta 1.0e+3.

C_range = np.logspace(-2, 10, num=13)
gamma_range = np.logspace(-9, 3, num=13)

param_grid_svm = dict(gamma=gamma_range, C=C_range)
nested_cv = 5

grid_svm = GridSearchCV(SVC(), param_grid=param_grid_svm, cv=nested_cv)

In [26]:
# Aquí se muestra el rango de valores a considerar
C_range,gamma_range

(array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
        1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10]),
 array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02,
        1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]))

In [27]:
'''
Definición del espacio de búsqueda para MLP
'''
alpha_range = np.logspace(-5, -1, 5)
hidden_layer_sizes_range=[(50,),(100,),(200,),(500,),(1000,)]

param_grid_mlp = dict(alpha=alpha_range, hidden_layer_sizes=hidden_layer_sizes_range)


grid_mlp = GridSearchCV(MLPClassifier(max_iter=1000,
                                      early_stopping=True), param_grid=param_grid_mlp, cv=nested_cv)

In [28]:
'''
Conjunto de clasificadores usados, así como sus nombres.
'''

cls_names = ["SVM","MLP"]

classifiers = [
    make_pipeline(StandardScaler(), grid_svm),
    make_pipeline(StandardScaler(), grid_mlp)]

In [29]:
# Método que ejecuta los clasificacodres y devuelve las etiquetas predichas correspondientes.

from sklearn.model_selection import train_test_split

def predictions(model,X_train,y_train,X_test,y_test):    
    
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    return y_test, y_pred

In [30]:
def predictions_model(X_train,y_train,X_test, y_test,model):
        '''
        Predicciones con un modelo y un conjunto de datos (X e y), para obtener posteriormente las medidas que se quieren
        
        Parámetros
        ----------
        X: numpy.array
            Conjunto (características)
        Y: numpy.array
            Dataset (etiquetas)
        model: scikit_model
            modelo a entrenar
        num_folds: int
            Número de "particiones" de la validación cruzada ("k-fold" cross validation)
        
        Devuelve
        -------
        array 
            array de predicciones
        '''
        print('\t'+str(model)[:20], end=' - ')
        y_test,preds = predictions(model,X_train,y_train,X_test,y_test)
        print('OK')
        
        return y_test,preds

In [32]:
from sklearn.model_selection import train_test_split

def run_all_save(filename):
    '''
    Realiza la validación cruzada de todos los modelos y conjuntos de datos.
        
        
    Parámetros
    ----------
    num_folds: int
        Igual que antes
    filename: string
        Nombre del fichero que guardará las "predicciones"
        
        
    El par X_train, y_train son los atributos y clases del conjunto de entrenamiento (70% de los ejemplos)
    El par X_test, y_test son los atributos y clases del conjunto de test (30% de los ejemplos)

    stratify (estratificar) significa que se quiere que haya la misma proporcion de cada una de las clases
    tanto en entrenamiento como en test, es decir, no es una partición completamente aleatoria.
    
    ''' 
    
    all_preds = {}

    for dataset,dataset_name in zip(datasets, dataset_names):
        print(dataset_name)
        X,y = dataset
        
        
        # TENÉIS QUE COMPLETAR AQUÍ
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)
        

        for model,cls_name in zip(classifiers,cls_names):
            print(cls_name)
            y_test,preds = predictions_model(X_train,y_train,X_test,y_test,model)
            all_preds[(dataset_name,cls_name)]=(y_test,preds)

    all_preds["cls_names"]=cls_names
    all_preds["dataset_names"]=dataset_names

    with open(filename, 'wb') as fp:
         pickle.dump(all_preds, fp)   

In [33]:
'''
All the predictions are going to be saved in a Python dictionary for 
further analysis.
'''

filename = 'PrediccionesFetalHealth.obj'

In [34]:
# Run the experiments

run_all_save(filename)

Data All
SVM
	Pipeline(steps=[('st - OK
MLP
	Pipeline(steps=[('st - OK


# Análisis de los resultados

Si los experimentos se han realizado previamente, sólo es necesario ejecutar el *notebook* desde esta parte. 

Los resultados se *cargarían* desde el disco duro.

In [37]:
import pickle
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

In [39]:
# Función que debe evaluar los resultados de clasificación.

def evalua(y_test, y_pred):
    
    return accuracy_score(y_test, y_pred)

In [40]:
def conf_mat_df(cm,labels):
    '''
    Creación de una matriz de confusión en un DataFrame
        
        
    Parámetros
    ----------
    cm: ndarray 2D
        matriz de confusión
    labels: lista
        Lista de nombres de clase
        
    Return DataFrame
    -------
    
    ''' 

    return (pd.DataFrame(cm,index=labels, columns=labels)
          .rename_axis("actual")
          .rename_axis("predicted", axis=1))

In [41]:
def get_results(filename):
    '''
    Carga el fichero con las predicciones.
    Calcula la "accuracy", la matriz de confusión, y otras. 
        
        
    Parámetros
    ----------
    filename: string
        Nombre del fichero que guarda las predicciones
        
    Return
    diccionario
        Un diccionario de pares key:values
    -------
    
    ''' 

    with open(filename, 'rb') as fp:
        all_preds = pickle.load(fp)

    cls_names = all_preds.pop("cls_names")
    dataset_names = all_preds.pop("dataset_names")

    data_cls_pairs = list(all_preds.keys())
    data_cls_pairs.sort()

    results = {}


    acc_df = pd.DataFrame(index=dataset_names, columns=cls_names)

    ## A DataFrame is created to store the accuracy in each clase
    for dataset in dataset_names:
        results[(dataset,"acc")] = pd.DataFrame(columns=cls_names)


    for dataset_name,cls_name in data_cls_pairs:

        #print(dataset_name,cls_name)
        y_true, y_pred = all_preds[(dataset_name,cls_name)]
        labels = list(np.unique(y_true))

        acc = evalua(y_true, y_pred)
        # Fill accuracy dataframe
        acc_df.at[dataset_name,cls_name]=acc

        # Get conf_mat
        cm = confusion_matrix(y_true, y_pred)
        cm_df = conf_mat_df(cm,labels)
        results[(dataset_name,cls_name,"cm")] = cm_df
        
        # Get classification report
        report = classification_report(y_true, y_pred, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        results[(dataset_name,cls_name,"report")] = report_df

        # Acc per class
        cm_dig = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_dig = cm_dig.diagonal()

        dfi = results[(dataset_name,"acc")]
        dfi[cls_name]=pd.Series(cm_dig,labels)    
        results[(dataset_name,"acc")]=dfi.copy()


    results["Acc"] = acc_df
    return results
        
        
results = get_results(filename)

In [42]:
df_total = results["Acc"].astype(float)
df_conf = results[("Data All","SVM","cm")].astype(float)
df_report = results[("Data All","SVM","report")].astype(float)

In [43]:
df_total

,SVM,MLP
Data All,1.0,0.984326


In [44]:
df_conf

predicted,1,2,3
actual,,,
1,500.0,0.0,0.0
2,0.0,90.0,0.0
3,0.0,0.0,48.0


In [24]:
df_report.round(4)[["precision","recall","f1-score"]]

,precision,recall,f1-score
1,0.9426,0.9710,0.9566
2,0.7680,0.6531,0.7059
3,0.8235,0.7955,0.8092
accuracy,0.9125,0.9125,0.9125
macro avg,0.8447,0.8065,0.8239
weighted avg,0.9086,0.9125,0.9097


---
## Análisis posteriores

En este apartado vas a tener que buscar información que te permita obtener lo que se pregunta en los tres puntos siguientes:

1. Estudia el efecto que tiene en la tasa de clasificación diferentes tipos de porcentajes de partición del conjunto, en conjunto de *entrenamiento* y de *test*. Puedes considerar, por ejemplo, los siguientes (normenclatura: (entrenamiento-test))

    - $50\%-50\%$
    - $60\%-40\%$
    - $70\%-30\%$
    - $80\%-20\%$


2. Haz una representación gráfica con el valor de la tasa de acierto en función de diferentes porcentajes de partición del conjunto


3. Programa la forma de ejecutar los dos métodos de clasificación, de tal forma que se ejecuten ambos $10$ veces, y se muestre la representación gráfica del punto $2$, pero en este caso considerando su valor medio y desviación estándar.

    - Un ejemplo de gráfica en la que se representaría la media y la desviación estándar sería de la siguiente forma:

    <img src="pics/GraficosBarrasError.png" width="50%">